<a href="https://colab.research.google.com/github/zloelias/final-project/blob/master/attack_textattack_from_github_character_level_ag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#! pip install tensorflow_text
#! pip install textattack['tensorflow']
#! pip install datasets --upgrade
#! pip install captum

In [3]:
# При установке из репозитория при работе не подцепляются другие модели, работает только дефолтный bert-base-uncased

! pip install tensorflow_text
! git clone https://github.com/zloelias/TextAttack.git
! pip install /content/TextAttack[tensorflow] --use-feature=in-tree-build
! pip install datasets --upgrade
! pip install captum

     |████████████████████████████████| 4.9 MB 15.5 MB/s 
Cloning into 'TextAttack'...
remote: Enumerating objects: 20340, done.
remote: Counting objects: 100% (2625/2625), done.
remote: Compressing objects: 100% (977/977), done.
remote: Total 20340 (delta 1782), reused 2334 (delta 1647), pack-reused 17715
Receiving objects: 100% (20340/20340), 25.42 MiB | 11.10 MiB/s, done.
Resolving deltas: 100% (15055/15055), done.
Processing ./TextAttack
     |████████████████████████████████| 60 kB 5.4 MB/s 
     |████████████████████████████████| 322 kB 29.3 MB/s 
     |████████████████████████████████| 769 kB 64.8 MB/s 
     |████████████████████████████████| 298 kB 80.8 MB/s 
     |████████████████████████████████| 3.3 MB 73.7 MB/s 
     |████████████████████████████████| 101 kB 13.1 MB/s 
     |████████████████████████████████| 454.3 MB 16 kB/s 
     |████████████████████████████████| 124 kB 84.2 MB/s 
     |████████████████████████████████| 462 kB 84.5 MB/s 
     |████████████████████████████

In [4]:
import torch
from copy import deepcopy

from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.models.wrappers import ModelWrapper
from transformers import AutoModelForSequenceClassification, AutoTokenizer

#from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients, LayerDeepLiftShap, InternalInfluence, LayerGradientXActivation
#from captum.attr import visualization as viz

import nltk
nltk.download('averaged_perceptron_tagger_ru')
nltk.download('stopwords')

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:16<00:00, 29.1MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpmow9qy7t.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else: 
    device = torch.device("cpu")
    
print(device)

In [6]:
from textattack import Attack
from textattack.constraints.pre_transformation import (
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder, MultilingualUniversalSentenceEncoder
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR, GreedySearch
from textattack.transformations import (
    CompositeTransformation,
    #WordSwapEmbedding,
    WordSwapHomoglyphSwap,
    WordSwapQWERTY,
    WordSwapNeighboringCharacterSwap,
    WordSwapRandomCharacterDeletion,
    WordSwapRandomCharacterInsertion,
)

from textattack.constraints.pre_transformation.max_word_index_modification import MaxWordIndexModification
from textattack.constraints.pre_transformation.max_modification_rate import MaxModificationRate

from textattack.attack_recipes.attack_recipe import AttackRecipe


In [7]:
class TextBuggerLi2018RU(AttackRecipe):
    """Li, J., Ji, S., Du, T., Li, B., and Wang, T. (2018).
    TextBugger: Generating Adversarial Text Against Real-world Applications.
    https://arxiv.org/abs/1812.05271
    """

    @staticmethod
    def build(model_wrapper, max_rate, max_seq_len, language):
        #
        #  we propose five bug generation methods for TEXTBUGGER:
        #
        transformation = CompositeTransformation(
            [
                # (1) Insert: Insert a space into the word.
                # Generally, words are segmented by spaces in English. Therefore,
                # we can deceive classifiers by inserting spaces into words.
                WordSwapRandomCharacterInsertion(
                    random_one=True,
                    letters_to_insert=" ",
                    skip_first_char=True,
                    skip_last_char=True,
                ),
                # (2) Delete: Delete a random character of the word except for the first
                # and the last character.
                WordSwapRandomCharacterDeletion(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
                # (3) Swap: Swap random two adjacent letters in the word but do not
                # alter the first or last letter. This is a common occurrence when
                # typing quickly and is easy to implement.
                WordSwapNeighboringCharacterSwap(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
                # (4) Substitute-C (Sub-C): Replace characters with visually similar
                # characters (e.g., replacing “o” with “0”, “l” with “1”, “a” with “@”)
                # or adjacent characters in the keyboard (e.g., replacing “m” with “n”).
                WordSwapHomoglyphSwap(),
                # (5) Substitute-W
                # (Sub-W): Replace a word with its topk nearest neighbors in a
                # context-aware word vector space. Specifically, we use the pre-trained
                # GloVe model [30] provided by Stanford for word embedding and set
                # topk = 5 in the experiment.
                #WordSwapEmbedding(max_candidates=5),
                WordSwapQWERTY(random_one=True, skip_first_char=True, skip_last_char=True),
            ]
        )

        constraints = [RepeatModification(), StopwordModification(language=language)]
        constraints.append(MaxWordIndexModification(max_seq_len))
        constraints.append(MaxModificationRate(max_rate, min_threshold=1))

        # In our experiment, we first use the Universal Sentence
        # Encoder [7], a model trained on a number of natural language
        # prediction tasks that require modeling the meaning of word
        # sequences, to encode sentences into high dimensional vectors.
        # Then, we use the cosine similarity to measure the semantic
        # similarity between original texts and adversarial texts.
        # ... "Furthermore, the semantic similarity threshold \eps is set
        # as 0.8 to guarantee a good trade-off between quality and
        # strength of the generated adversarial text."
        constraints.append(MultilingualUniversalSentenceEncoder(
            threshold=0.936338023,
            metric="cosine",
            compare_against_original=True,
            window_size=15,
            skip_text_shorter_than_window=True,
        ))
        #
        # Goal is untargeted classification
        #
        goal_function = UntargetedClassification(model_wrapper)
        #
        # Greedily swap words with "Word Importance Ranking".
        #
        search_method = GreedyWordSwapWIR(wir_method="delete")

        return Attack(goal_function, constraints, transformation, search_method)




In [9]:
import textattack
import random
import numpy as np

experiments = {
    'simple_char_level_attack': [
#        {
#            'dataset': {'name': 'kinopoisk', 'url': 'zloelias/kinopoisk-reviews'},
#            'model': ('cointegrated/rubert-tiny2', 'zloelias/rubert-tiny2-kinopoisk-reviews-finetuned-clf'),
#            'language': ('ru', 'rus', 'russian'),
#            'max_rate': np.linspace(0.1, 0.9, 9),
#            'max_seq_len': 312,
#            'columns': (['text'], 'labels'),
#        },
#        {
#            'dataset': {'name': 'lenta', 'url': 'zloelias/lenta-ru'},
#            'model': ('cointegrated/rubert-tiny2', 'zloelias/rubert-tiny2-lenta-ru-finetuned-clf'),
#            'language': ('ru', 'rus', 'russian'),
#            'max_rate': np.linspace(0.1, 0.9, 9),
#            'max_seq_len': 312,
#            'columns': (['text'], 'labels'),
#        },
        {
            'dataset': {'name': 'ag_news', 'url': 'ag_news'},
            'model': ('bert-base-uncased', 'textattack/bert-base-uncased-ag-news'),
            'language': ('en', 'eng', 'english'),
            'max_rate': np.linspace(0.1, 0.9, 9),
            'max_seq_len': 512,
            'columns': (['text'], 'label'),
        },
 #       {
 #           'dataset': {'name': 'rotten_tomatoes', 'url': 'rotten_tomatoes'},
 #           'model': ('bert-base-uncased', 'textattack/bert-base-uncased-rotten-tomatoes'),
 #           'language': ('en', 'eng', 'english'),
 #           'max_rate': np.linspace(0.1, 0.9, 9),
 #           'max_seq_len': 512,
 #           'columns': (['text'], 'label'),
 #       },
    ]
}

for attack_name in experiments.keys():
    for attack_data in experiments[attack_name]:
        dataset_name, dataset_url = [attack_data['dataset'][k] for k in attack_data['dataset'].keys()]
        dataset_columns = attack_data['columns']
        dataset = HuggingFaceDataset(dataset_url, None, 'test', dataset_columns=dataset_columns)

        base_model_name, model_name = attack_data['model']
        original_model = AutoModelForSequenceClassification.from_pretrained(model_name)
        original_tokenizer = AutoTokenizer.from_pretrained(base_model_name)
        model = HuggingFaceModelWrapper(original_model,original_tokenizer)

        la, lan, language = attack_data['language']
        for max_rate in attack_data['max_rate']:
            experiment_result_dir = f'/content/drive/MyDrive/Colab Notebooks/final-project/experiments/{attack_name}/{dataset_name}/{max_rate}/'
            print(experiment_result_dir)
            attack = TextBuggerLi2018RU.build(model, max_rate, attack_data['max_seq_len'], language)

            attack_args = textattack.AttackArgs(
                num_examples=300, 
                log_to_csv=experiment_result_dir + "log.csv", 
                checkpoint_interval=50, 
                checkpoint_dir=experiment_result_dir + "checkpoints", 
                disable_stdout=True
            )
            attacker = textattack.Attacker(attack, dataset, attack_args)
            attacker.attack_dataset()
        #print(attack_data)
        #print(model_name, model_url)

Using custom data configuration default


Downloading:   0%|          | 0.00/11.0M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/751k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

...

[Succeeded / Failed / Skipped / Total] 39 / 104 / 7 / 150:  50%|█████     | 150/300 [03:43<03:43,  1.49s/it]textattack: Saving checkpoint under "/content/drive/MyDrive/Colab Notebooks/final-project/experiments/simple_char_level_attack/ag_news/0.9/checkpoints/1639417658207.ta.chkpt" at 2021-12-13 17:47:38 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 50 / 143 / 7 / 200:  67%|██████▋   | 200/300 [05:01<02:30,  1.51s/it]textattack: Saving checkpoint under "/content/drive/MyDrive/Colab Notebooks/final-project/experiments/simple_char_level_attack/ag_news/0.9/checkpoints/1639417736356.ta.chkpt" at 2021-12-13 17:48:56 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 60 / 178 / 12 / 250:  83%|████████▎ | 250/300 [06:14<01:14,  1.50s/it]textattack: Saving checkpoint under "/content/drive/MyDrive/Colab Notebooks/final-project/experiments/simple_char_level_attack/ag_news/0.9/checkpoints/1639417809465.ta.chkpt" at 2021-12-13 17:50:09 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 78 / 207 / 15 / 300: 100%|██████████| 300/300 [07:19<00:00,  1.46s/it]textattack: Saving checkpoint under "/content/drive/MyDrive/Colab Notebooks/final-project/experiments/simple_char_level_attack/ag_news/0.9/checkpoints/1639417874542.ta.chkpt" at 2021-12-13 17:51:14 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 78 / 207 / 15 / 300: 100%|██████████| 300/300 [07:19<00:00,  1.46s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 78     |
| Number of failed attacks:     | 207    |
| Number of skipped attacks:    | 15     |
| Original accuracy:            | 95.0%  |
| Accuracy under attack:        | 69.0%  |
| Attack success rate:          | 27.37% |
| Average perturbed word %:     | 18.84% |
| Average num. words per input: | 41.2   |
| Avg num queries:              | 118.54 |
+-------------------------------+--------+
